In [217]:
from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np 
from scipy.stats import t

### Notebook for the evaluation of the audio quality survey

See the web application for collecting the data on the repository https://github.com/TSHP/PFGM_MOS.git.

In [218]:

feedbacks = [f for f in listdir("feedback_files") if isfile(join("feedback_files", f))]

df = pd.DataFrame(columns=["round", "file", "score", "id"])
for feedback in feedbacks:
    temp_df = pd.read_csv("feedback_files/" + feedback)
    temp_df["id"] = [feedback.split(".")[0]] * len(temp_df) 
    # Only consider surveys with at least 100 answers
    if len(temp_df) == 121:
        df = df.append(temp_df[:120])
    elif len(temp_df) >= 100:
        df = df.append(temp_df)
    if len(temp_df) < 100:
        continue

df = df.drop(labels=["Unnamed: 0"], axis=1)
df = df[df["score"] != "Unable to Assign a Score"]
df["model"] = df["file"].apply(lambda x: x.split("/")[1])
df["score"] = df["score"].astype(float)

/var/folders/js/cmc_gtgj52l8j7qm2wtz53kc0000gn/T/ipykernel_46056/4106297164.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df[:120])
/var/folders/js/cmc_gtgj52l8j7qm2wtz53kc0000gn/T/ipykernel_46056/4106297164.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df[:120])
/var/folders/js/cmc_gtgj52l8j7qm2wtz53kc0000gn/T/ipykernel_46056/4106297164.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df[:120])
/var/folders/js/cmc_gtgj52l8j7qm2wtz53kc0000gn/T/ipykernel_46056/4106297164.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
/var/folders/js/c

### Compute mean and 95% confidence intervals for all the models

In [230]:
grouped = df.groupby(['id', 'file', 'model'])
mean_by_id_and_file = grouped["score"].mean()

x = len(df["id"].value_counts()) * 10
dof = x - 1 
confidence = 0.95

t_crit = np.abs(t.ppf((1 - confidence) / 2, dof))
means = mean_by_id_and_file.groupby("model").mean()
print(means)
stds = mean_by_id_and_file.groupby("model").std()
print(stds)

for m, s, model_name in zip(means, stds, means.index):
    print(f"{model_name}: ({-s*t_crit/np.sqrt(x)}, {s*t_crit/np.sqrt(x)})")

model
diffwave       3.331373
pfgm_128       3.549020
pfgm_128_sd    2.839216
wavegan        1.901961
Name: score, dtype: float64
model
diffwave       1.236896
pfgm_128       1.075342
pfgm_128_sd    1.214759
wavegan        0.708405
Name: score, dtype: float64
diffwave: (-0.18727414455395644, 0.18727414455395644)
pfgm_128: (-0.16281383750401246, 0.16281383750401246)
pfgm_128_sd: (-0.18392250026795487, 0.18392250026795487)
wavegan: (-0.10725712137793304, 0.10725712137793304)
